<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/04_SRTM90_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process DEM to derive topographical indices


## Import gee and authenticate

Expiry period: the granted permissions will expire in a week, after which you'll need to call ee.Authenticate() again.

In [ ]:
# This doesn't really work, it's only valid for the current runtime...
import os

# If credentials file doesn't exist, authenticate and store credentials
# Else if credentials file does exist, use stored credentials and initialise
if not os.path.exists(os.path.expanduser("~/.config/earthengine/credentials")):
  import ee
  ee.Authenticate()
  ee.Initialize()
else:
  import ee
  ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=lc9qnatIuR90-U6hO7X4ajej7R81jl-ngWQrSwZtQCc&tc=i-h6NuuiXDGvVY2GJ1BSqrxYCWSKs6g2DtOOoWKeC9I&cc=kOIA1buMI0vX5lE_EDM-VYX2gs8row-8fEvRZCAeTy4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VMZttUAD_W4DaBp_z61gMKvhk7u3NhaNLm9c3LFXpgy0f5_qUUJqKs

Successfully saved authorization token.


## Import AOI and set variable names


In [ ]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_PACE")
aoi_name = "MKR_PACE"

elev_layer_name = "RS_005"
slope_layer_name = "RS_006"
aspect_layer_name = "RS_007"

## Import image and calculate topography metrics


In [ ]:
srtm = ee.Image("CGIAR/SRTM90_V4")
elevation = srtm.select("elevation").clip(aoi)
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

## Export data - create tasks

In [ ]:
task1 = ee.batch.Export.image.toDrive(image = elevation,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = elev_layer_name + "_" + aoi_name,
                                     scale = 30,
                                     maxPixels = 10e12
                                     )
task1.start()

In [ ]:
task2 = ee.batch.Export.image.toDrive(image = slope,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = slope_layer_name + "_" + aoi_name,
                                     scale = 30,
                                     maxPixels = 10e12
                                     )
task2.start()

In [ ]:
task3 = ee.batch.Export.image.toDrive(image = aspect,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = aspect_layer_name + "_" + aoi_name,
                                     scale = 30,
                                     maxPixels = 10e12
                                     )

task3.start()

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:3]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

NXQNREFOQLDFDIXG4GPR4UTS READY
XVG4NDBAH7RGP7GDCZDR6GPG READY
AWIJ6IHDLQTVAFGLRWQ27KQW READY
